In [191]:
import sys


!{sys.executable} -m pip install nltk



In [265]:
import pandas as pd

import csv
import re # for regular expression
import string
from datetime import datetime

import nltk 
from nltk.tokenize import RegexpTokenizer # devide the words 
from collections import Counter #count the number of word for each list 

import numpy as np 
import matplotlib.pyplot as plt

import seaborn as sns


#import plotly
#import plotly.plotly as py
#import plotly.graph_objs as go
#plotly.tools.set_credentials_file(username='XXXXXX', api_key='XXXXXXX')

%matplotlib inline

In [193]:
!pwd

'pwd' is not recognized as an internal or external command,
operable program or batch file.


In [194]:
# read csv file and save it in a data frame
path = '/Users/lama/Desktop/SWE485-Phase3/'
df=pd.read_csv('CleanedData.csv') 

In [195]:
# how much data do I have?
len(df)

881

In [196]:
print(sum(df.duplicated()))

0


In [197]:
# Any missing values?
df.text.isnull().values.any()

False

In [198]:
# Total missing values for each feature
df.isnull().sum()

Unnamed: 0    0
replies       0
retweets      0
favorites     0
text          0
day           0
time          0
dtype: int64

In [199]:
df =df.drop(columns="Unnamed: 0")

In [200]:
# remove Arabic stop words from the tweets
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
ar_stops = set(stopwords.words('arabic'))



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jwoha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [201]:
print (ar_stops)

{'كليكما', 'لكن', 'كي', 'لو', 'اللتان', 'تلكم', 'ماذا', 'اللتين', 'أنتم', 'بعد', 'وما', 'عند', 'بعض', 'به', 'بي', 'أين', 'أقل', 'إما', 'ته', 'لستن', 'لئن', 'الذين', 'قد', 'إن', 'منذ', 'هذه', 'كيت', 'ذينك', 'لولا', 'هنا', 'إليكم', 'التي', 'فإن', 'أيها', 'ولكن', 'نعم', 'هاتي', 'ذان', 'أنتن', 'ذاك', 'كلما', 'أينما', 'لست', 'هاتان', 'كليهما', 'لك', 'أو', 'ممن', 'بخ', 'على', 'والذي', 'أف', 'لهم', 'كأنما', 'دون', 'ذي', 'هلا', 'إلى', 'كلتا', 'يا', 'ذوا', 'هيهات', 'وإن', 'أنتما', 'فيها', 'آه', 'فيما', 'عل', 'ومن', 'أوه', 'إنما', 'ذين', 'بيد', 'متى', 'هو', 'في', 'بنا', 'اللذين', 'شتان', 'عليك', 'حبذا', 'هؤلاء', 'ريث', 'بهن', 'هما', 'بس', 'هذان', 'نحن', 'ذلكن', 'كما', 'ذو', 'هذي', 'ليسا', 'ولا', 'مذ', 'كذا', 'ذلك', 'هناك', 'ذواتي', 'مما', 'اللائي', 'ذلكم', 'سوى', 'اللاتي', 'بك', 'ليست', 'تلك', 'لا', 'وإذ', 'مه', 'أما', 'كأن', 'فإذا', 'كيف', 'عسى', 'لعل', 'عدا', 'آي', 'كذلك', 'هن', 'بهم', 'لم', 'هيا', 'بماذا', 'أنى', 'بلى', 'ثم', 'الذي', 'تلكما', 'حيثما', 'إيه', 'عما', 'إذما', 'إنه', 'إذن', 'كم',

In [202]:
print(df['text'])

0      وماذا عن إزعاج ما يزيد على ستة أشهر لأحياء سكن...
1                                         عدلك يا رب    
2         إلا قرقعة ودوشة قارفينا ببيتي قال فن قال      
3      ٦ شهور محبورة اعيش ببيتي والبوليفارد يردح على ...
4      في شويه معرصين حطين الصورة دي  انا مش مطالبه ا...
                             ...                        
876      أتمنى منك عرض معاناة منطقة جازان في توسعة مل...
877      صدقني أحس إن حطين بالدرجة الثانية ماله حس و ...
878      مع ان جيرانا من حطين ولوبية وصفورية والمرين ...
879    اليوم الطائي يحل على حطين ضيف ولسان حال العشاق...
880    نبارك لأهالي حطين الكرام افتتاح جمعيتنا بعد مع...
Name: text, Length: 881, dtype: object


In [203]:
# remove Arabic stop words from the tweets
clean_text = []
for row in df['text']:
    word_list = row.split()
    word_list = [word for word in word_list if word not in ar_stops]
    clean_text.append(word_list)

In [204]:
print(clean_text)

[['وماذا', 'إزعاج', 'يزيد', 'ستة', 'أشهر', 'لأحياء', 'سكنية', 'وآلاف', 'السكان', 'ومئات', 'الأسر', 'وعشرات', 'المساجد', 'جاورها', 'معاناة', 'ومأساة', 'وظلم', 'يزال', 'التفات', 'رفع', 'استجابة'], ['عدلك', 'رب'], ['قرقعة', 'ودوشة', 'قارفينا', 'ببيتي', 'قال', 'فن', 'قال'], ['٦', 'شهور', 'محبورة', 'اعيش', 'ببيتي', 'والبوليفارد', 'يردح', 'راسي', 'مشاركنا', 'بيتنا', 'اش', 'هالتجني', 'وعدم', 'احترام', 'حرمات', 'الناس', 'الله', 'يخوي', 'روسهم', 'مثل', 'اخووا', 'روسنا', 'بسماعاتهم', 'طوال', 'اليوم'], ['شويه', 'معرصين', 'حطين', 'الصورة', 'دي', 'انا', 'مش', 'مطالبه', 'اقعد', 'اشرح', 'معاناة', 'اللي', 'تعبنا', 'واحنا', 'بنشرحها', 'وانتوا', 'خنزير', 'ناطقه', 'مبتفهمش', 'اي', 'حد', 'حاطط', 'الصورة', 'دي', 'احب', 'اقوله', 'اسكت', 'ياعر', 'ص', 'ء'], ['حسبي', 'الله', 'ونعم', 'الوكيل', 'عدلك', 'رب', 'العالمين'], ['حلول', 'المرور', 'مشاكل', 'يقفلون', 'اليوتيرن', 'اللي', 'اشارة', 'تركي', 'الأول', 'يقال', 'بيخففون', 'الزحمه', 'والحاصل', 'زحمه', 'ممتده', 'حطين', 'ودي', 'اوقف', 'السياره', 'وارجع', 'البيت', '

In [205]:
#df['clean_text'] = clean_text
print(len(clean_text))

881


In [206]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 881 entries, 0 to 880
Data columns (total 6 columns):
replies      881 non-null int64
retweets     881 non-null int64
favorites    881 non-null int64
text         881 non-null object
day          881 non-null object
time         881 non-null object
dtypes: int64(3), object(3)
memory usage: 31.0+ KB


In [207]:
df.head(20)

,replies,retweets,favorites,text,day,time
0,0,0,0,وماذا عن إزعاج ما يزيد على ستة أشهر لأحياء سكن...,02/08/2020,09:16
1,0,0,0,عدلك يا رب,02/04/2020,22:03
2,1,0,0,إلا قرقعة ودوشة قارفينا ببيتي قال فن قال,24/01/2020,20:02
3,1,0,0,٦ شهور محبورة اعيش ببيتي والبوليفارد يردح على ...,21/01/2020,21:03
4,0,0,0,في شويه معرصين حطين الصورة دي انا مش مطالبه ا...,19/01/2020,21:59
5,1,0,0,حسبي الله ونعم الوكيل عدلك يا رب العالمين,18/01/2020,20:01
6,0,0,0,حلول المرور مشاكل يقفلون اليوتيرن اللي عند اشا...,16/01/2020,20:50
7,1,0,0,عدلك يا رب في المسؤول عن هذا الازعاج و السماعا...,13/01/2020,20:17
8,1,0,0,اتمنى نلقى حل لهذي المشكلة جدا جدا جدا متعب ال...,01/12/2020,22:22
9,0,0,0,اقول روقو يااهل حطين المتر عندكم ب ٥٠٠٠الآف ...,01/01/2020,10:51


In [208]:
df['tokens'] = clean_text

In [209]:
df

,replies,retweets,favorites,text,day,time,tokens
0,0,0,0,وماذا عن إزعاج ما يزيد على ستة أشهر لأحياء سكن...,02/08/2020,09:16,"[وماذا, إزعاج, يزيد, ستة, أشهر, لأحياء, سكنية,..."
1,0,0,0,عدلك يا رب,02/04/2020,22:03,"[عدلك, رب]"
2,1,0,0,إلا قرقعة ودوشة قارفينا ببيتي قال فن قال,24/01/2020,20:02,"[قرقعة, ودوشة, قارفينا, ببيتي, قال, فن, قال]"
3,1,0,0,٦ شهور محبورة اعيش ببيتي والبوليفارد يردح على ...,21/01/2020,21:03,"[٦, شهور, محبورة, اعيش, ببيتي, والبوليفارد, ير..."
4,0,0,0,في شويه معرصين حطين الصورة دي انا مش مطالبه ا...,19/01/2020,21:59,"[شويه, معرصين, حطين, الصورة, دي, انا, مش, مطال..."
...,...,...,...,...,...,...,...
876,1,0,0,أتمنى منك عرض معاناة منطقة جازان في توسعة مل...,12/05/2012,19:35,"[أتمنى, منك, عرض, معاناة, منطقة, جازان, توسعة,..."
877,0,0,0,صدقني أحس إن حطين بالدرجة الثانية ماله حس و ...,12/01/2012,20:46,"[صدقني, أحس, حطين, بالدرجة, الثانية, ماله, حس,..."
878,0,0,0,مع ان جيرانا من حطين ولوبية وصفورية والمرين ...,18/09/2012,00:53,"[ان, جيرانا, حطين, ولوبية, وصفورية, والمرين, ف..."
879,0,0,0,اليوم الطائي يحل على حطين ضيف ولسان حال العشاق...,09/12/2012,13:10,"[اليوم, الطائي, يحل, حطين, ضيف, ولسان, حال, ال..."


In [210]:
#Replace Letters
"""
for tokens in clean_text: 
    for word in tokens:
        word = word.str.replace('أ','ا')
        word = word.str.replace('إ','ا')
        word = word.str.replace('ة','ه')
"""
# Function to convert   
def listToString(s):  
    
    # initialize an empty string 
    new_list = []   
    flag = False
    # traverse in the string   
    for ele in s:
        if flag:
            new_list.append(str1)
        str1 = "" 
        for e in ele:
            flag= True
            str1+= ' '+e
            
    ###### for the last element ########
    for ele in s:
        str1 = "" 
        for e in ele:
            str1+= ' '+e
            
    new_list.append(str1)
    print(new_list)
    # return string   
    return new_list  

list_string = listToString(clean_text)



[' وماذا إزعاج يزيد ستة أشهر لأحياء سكنية وآلاف السكان ومئات الأسر وعشرات المساجد جاورها معاناة ومأساة وظلم يزال التفات رفع استجابة', ' عدلك رب', ' قرقعة ودوشة قارفينا ببيتي قال فن قال', ' ٦ شهور محبورة اعيش ببيتي والبوليفارد يردح راسي مشاركنا بيتنا اش هالتجني وعدم احترام حرمات الناس الله يخوي روسهم مثل اخووا روسنا بسماعاتهم طوال اليوم', ' شويه معرصين حطين الصورة دي انا مش مطالبه اقعد اشرح معاناة اللي تعبنا واحنا بنشرحها وانتوا خنزير ناطقه مبتفهمش اي حد حاطط الصورة دي احب اقوله اسكت ياعر ص ء', ' حسبي الله ونعم الوكيل عدلك رب العالمين', ' حلول المرور مشاكل يقفلون اليوتيرن اللي اشارة تركي الأول يقال بيخففون الزحمه والحاصل زحمه ممتده حطين ودي اوقف السياره وارجع البيت مشي', ' عدلك رب المسؤول الازعاج و السماعات المسعورة الله يجعل حوبة ازعاج الحي منازلهم تتعاداه صحته و رزقه حسبي الله ونعم الوكيل', ' اتمنى نلقى حل لهذي المشكلة جدا جدا جدا متعب الازعاج والصوت الصاخب مو قادرين ننام الليل اضبطو مستوى الصوت وتكون رقابة قويه اذا مسؤولين الصوت يبالغون اتمنى القى حل لهذي المشكلة وشكرا', ' اقول روقو 

In [211]:
df['normalized'] = list_string

In [212]:
df

,replies,retweets,favorites,text,day,time,tokens,normalized
0,0,0,0,وماذا عن إزعاج ما يزيد على ستة أشهر لأحياء سكن...,02/08/2020,09:16,"[وماذا, إزعاج, يزيد, ستة, أشهر, لأحياء, سكنية,...",وماذا إزعاج يزيد ستة أشهر لأحياء سكنية وآلاف ...
1,0,0,0,عدلك يا رب,02/04/2020,22:03,"[عدلك, رب]",عدلك رب
2,1,0,0,إلا قرقعة ودوشة قارفينا ببيتي قال فن قال,24/01/2020,20:02,"[قرقعة, ودوشة, قارفينا, ببيتي, قال, فن, قال]",قرقعة ودوشة قارفينا ببيتي قال فن قال
3,1,0,0,٦ شهور محبورة اعيش ببيتي والبوليفارد يردح على ...,21/01/2020,21:03,"[٦, شهور, محبورة, اعيش, ببيتي, والبوليفارد, ير...",٦ شهور محبورة اعيش ببيتي والبوليفارد يردح راس...
4,0,0,0,في شويه معرصين حطين الصورة دي انا مش مطالبه ا...,19/01/2020,21:59,"[شويه, معرصين, حطين, الصورة, دي, انا, مش, مطال...",شويه معرصين حطين الصورة دي انا مش مطالبه اقعد...
...,...,...,...,...,...,...,...,...
876,1,0,0,أتمنى منك عرض معاناة منطقة جازان في توسعة مل...,12/05/2012,19:35,"[أتمنى, منك, عرض, معاناة, منطقة, جازان, توسعة,...",أتمنى منك عرض معاناة منطقة جازان توسعة ملعب م...
877,0,0,0,صدقني أحس إن حطين بالدرجة الثانية ماله حس و ...,12/01/2012,20:46,"[صدقني, أحس, حطين, بالدرجة, الثانية, ماله, حس,...",صدقني أحس حطين بالدرجة الثانية ماله حس و مأكا...
878,0,0,0,مع ان جيرانا من حطين ولوبية وصفورية والمرين ...,18/09/2012,00:53,"[ان, جيرانا, حطين, ولوبية, وصفورية, والمرين, ف...",ان جيرانا حطين ولوبية وصفورية والمرين فقط علي...
879,0,0,0,اليوم الطائي يحل على حطين ضيف ولسان حال العشاق...,09/12/2012,13:10,"[اليوم, الطائي, يحل, حطين, ضيف, ولسان, حال, ال...",اليوم الطائي يحل حطين ضيف ولسان حال العشاق لل...


In [213]:
#Replace Letters

df['normalized'] = df['normalized'].str.replace('أ','ا')
df['normalized'] = df['normalized'].str.replace('إ','ا')
df['normalized'] = df['normalized'].str.replace('ة','ه')




In [214]:
df

,replies,retweets,favorites,text,day,time,tokens,normalized
0,0,0,0,وماذا عن إزعاج ما يزيد على ستة أشهر لأحياء سكن...,02/08/2020,09:16,"[وماذا, إزعاج, يزيد, ستة, أشهر, لأحياء, سكنية,...",وماذا ازعاج يزيد سته اشهر لاحياء سكنيه وآلاف ...
1,0,0,0,عدلك يا رب,02/04/2020,22:03,"[عدلك, رب]",عدلك رب
2,1,0,0,إلا قرقعة ودوشة قارفينا ببيتي قال فن قال,24/01/2020,20:02,"[قرقعة, ودوشة, قارفينا, ببيتي, قال, فن, قال]",قرقعه ودوشه قارفينا ببيتي قال فن قال
3,1,0,0,٦ شهور محبورة اعيش ببيتي والبوليفارد يردح على ...,21/01/2020,21:03,"[٦, شهور, محبورة, اعيش, ببيتي, والبوليفارد, ير...",٦ شهور محبوره اعيش ببيتي والبوليفارد يردح راس...
4,0,0,0,في شويه معرصين حطين الصورة دي انا مش مطالبه ا...,19/01/2020,21:59,"[شويه, معرصين, حطين, الصورة, دي, انا, مش, مطال...",شويه معرصين حطين الصوره دي انا مش مطالبه اقعد...
...,...,...,...,...,...,...,...,...
876,1,0,0,أتمنى منك عرض معاناة منطقة جازان في توسعة مل...,12/05/2012,19:35,"[أتمنى, منك, عرض, معاناة, منطقة, جازان, توسعة,...",اتمنى منك عرض معاناه منطقه جازان توسعه ملعب م...
877,0,0,0,صدقني أحس إن حطين بالدرجة الثانية ماله حس و ...,12/01/2012,20:46,"[صدقني, أحس, حطين, بالدرجة, الثانية, ماله, حس,...",صدقني احس حطين بالدرجه الثانيه ماله حس و ماكا...
878,0,0,0,مع ان جيرانا من حطين ولوبية وصفورية والمرين ...,18/09/2012,00:53,"[ان, جيرانا, حطين, ولوبية, وصفورية, والمرين, ف...",ان جيرانا حطين ولوبيه وصفوريه والمرين فقط علي...
879,0,0,0,اليوم الطائي يحل على حطين ضيف ولسان حال العشاق...,09/12/2012,13:10,"[اليوم, الطائي, يحل, حطين, ضيف, ولسان, حال, ال...",اليوم الطائي يحل حطين ضيف ولسان حال العشاق لل...


In [215]:
# remove punctuations علامات الترقيم  

arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)

In [216]:
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text

In [217]:
# remove repeated letters
def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)

In [218]:
#start processing the tweet
def processPost(tweet):

    #Replace @username with empty string
    tweet = re.sub('@[^\s]+', ' ', tweet)
    
    #Convert www.* or https?://* to " "
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',tweet)
    
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)

    # remove punctuations
    tweet= remove_punctuations(tweet)
    
    # normalize the tweet
    tweet= normalize_arabic(tweet)
    
    # remove repeated letters
    tweet=remove_repeating_char(tweet)

    #Remove new lines with white space
    tweet = tweet.replace('\n',' ')
    tweet = tweet.replace('\t',' ')
        
    return tweet

In [219]:
# apply the method in all the text and creat another cloumn with the processed text
df["normalized"] = df["normalized"].apply(lambda x: processPost(x))

In [220]:
# check the new column
df.head(20) # clean text column contains the text after applying all the preprocessing steps (i.e remove duplicated letters, new lines ..etc)

,replies,retweets,favorites,text,day,time,tokens,normalized
0,0,0,0,وماذا عن إزعاج ما يزيد على ستة أشهر لأحياء سكن...,02/08/2020,09:16,"[وماذا, إزعاج, يزيد, ستة, أشهر, لأحياء, سكنية,...",وماذا ازعاج يزيد سته اشهر لاحياء سكنيه والاف ...
1,0,0,0,عدلك يا رب,02/04/2020,22:03,"[عدلك, رب]",عدلك رب
2,1,0,0,إلا قرقعة ودوشة قارفينا ببيتي قال فن قال,24/01/2020,20:02,"[قرقعة, ودوشة, قارفينا, ببيتي, قال, فن, قال]",قرقعه ودوشه قارفينا بيتي قال فن قال
3,1,0,0,٦ شهور محبورة اعيش ببيتي والبوليفارد يردح على ...,21/01/2020,21:03,"[٦, شهور, محبورة, اعيش, ببيتي, والبوليفارد, ير...",٦ شهور محبوره اعيش بيتي والبوليفارد يردح راسي...
4,0,0,0,في شويه معرصين حطين الصورة دي انا مش مطالبه ا...,19/01/2020,21:59,"[شويه, معرصين, حطين, الصورة, دي, انا, مش, مطال...",شويه معرصين حطين الصوره دي انا مش مطالبه اقعد...
5,1,0,0,حسبي الله ونعم الوكيل عدلك يا رب العالمين,18/01/2020,20:01,"[حسبي, الله, ونعم, الوكيل, عدلك, رب, العالمين]",حسبي اله ونعم الوكيل عدلك رب العالمين
6,0,0,0,حلول المرور مشاكل يقفلون اليوتيرن اللي عند اشا...,16/01/2020,20:50,"[حلول, المرور, مشاكل, يقفلون, اليوتيرن, اللي, ...",حلول المرور مشاكل يقفلون اليوتيرن الي اشاره ت...
7,1,0,0,عدلك يا رب في المسؤول عن هذا الازعاج و السماعا...,13/01/2020,20:17,"[عدلك, رب, المسؤول, الازعاج, و, السماعات, المس...",عدلك رب المسؤول الازعاج و السماعات المسعوره ا...
8,1,0,0,اتمنى نلقى حل لهذي المشكلة جدا جدا جدا متعب ال...,01/12/2020,22:22,"[اتمنى, نلقى, حل, لهذي, المشكلة, جدا, جدا, جدا...",اتمني نلقي حل لهذي المشكله جدا جدا جدا متعب ا...
9,0,0,0,اقول روقو يااهل حطين المتر عندكم ب ٥٠٠٠الآف ...,01/01/2020,10:51,"[اقول, روقو, يااهل, حطين, المتر, عندكم, ب, ٥٠٠...",اقول روقو ياهل حطين المتر عندكم ب ٥٠الاف مايض...


In [221]:
export_csv = df.to_csv (r'HittenTweetsCleaned.csv', index = None, header=True, encoding='utf-8-sig')

In [222]:
#df = df.drop(columns=['clean text','text length','index_col'])

In [223]:
#df = df.reindex(columns=['index','clean_text','text_length','replies','retweets','favorites','day','time','text'])

In [224]:
print(len(df))

881


In [225]:
df = df[df.normalized != ''] #remove the empty tweets that are not detected by (dropna).

In [226]:
print(len(df))

813


In [227]:
df

,replies,retweets,favorites,text,day,time,tokens,normalized
0,0,0,0,وماذا عن إزعاج ما يزيد على ستة أشهر لأحياء سكن...,02/08/2020,09:16,"[وماذا, إزعاج, يزيد, ستة, أشهر, لأحياء, سكنية,...",وماذا ازعاج يزيد سته اشهر لاحياء سكنيه والاف ...
1,0,0,0,عدلك يا رب,02/04/2020,22:03,"[عدلك, رب]",عدلك رب
2,1,0,0,إلا قرقعة ودوشة قارفينا ببيتي قال فن قال,24/01/2020,20:02,"[قرقعة, ودوشة, قارفينا, ببيتي, قال, فن, قال]",قرقعه ودوشه قارفينا بيتي قال فن قال
3,1,0,0,٦ شهور محبورة اعيش ببيتي والبوليفارد يردح على ...,21/01/2020,21:03,"[٦, شهور, محبورة, اعيش, ببيتي, والبوليفارد, ير...",٦ شهور محبوره اعيش بيتي والبوليفارد يردح راسي...
4,0,0,0,في شويه معرصين حطين الصورة دي انا مش مطالبه ا...,19/01/2020,21:59,"[شويه, معرصين, حطين, الصورة, دي, انا, مش, مطال...",شويه معرصين حطين الصوره دي انا مش مطالبه اقعد...
...,...,...,...,...,...,...,...,...
876,1,0,0,أتمنى منك عرض معاناة منطقة جازان في توسعة مل...,12/05/2012,19:35,"[أتمنى, منك, عرض, معاناة, منطقة, جازان, توسعة,...",اتمني منك عرض معاناه منطقه جازان توسعه ملعب م...
877,0,0,0,صدقني أحس إن حطين بالدرجة الثانية ماله حس و ...,12/01/2012,20:46,"[صدقني, أحس, حطين, بالدرجة, الثانية, ماله, حس,...",صدقني احس حطين بالدرجه الثانيه ماله حس و ماكا...
878,0,0,0,مع ان جيرانا من حطين ولوبية وصفورية والمرين ...,18/09/2012,00:53,"[ان, جيرانا, حطين, ولوبية, وصفورية, والمرين, ف...",ان جيرانا حطين ولوبيه وصفوريه والمرين فقط علي...
879,0,0,0,اليوم الطائي يحل على حطين ضيف ولسان حال العشاق...,09/12/2012,13:10,"[اليوم, الطائي, يحل, حطين, ضيف, ولسان, حال, ال...",اليوم الطائي يحل حطين ضيف ولسان حال العشاق لر...


In [228]:
export_csv = df.to_csv (r'moreCleanedVersion.csv', index = None, header=True, encoding='utf-8-sig')

In [229]:
# remove Arabic stop words from the tweets
tokenized_list = []
for row in df['normalized']:
    word_list = row.split()
    tokenized_list.append(word_list)

In [230]:
df['tokenized'] = tokenized_list

C:\Users\jwoha\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [231]:
df

,replies,retweets,favorites,text,day,time,tokens,normalized,tokenized
0,0,0,0,وماذا عن إزعاج ما يزيد على ستة أشهر لأحياء سكن...,02/08/2020,09:16,"[وماذا, إزعاج, يزيد, ستة, أشهر, لأحياء, سكنية,...",وماذا ازعاج يزيد سته اشهر لاحياء سكنيه والاف ...,"[وماذا, ازعاج, يزيد, سته, اشهر, لاحياء, سكنيه,..."
1,0,0,0,عدلك يا رب,02/04/2020,22:03,"[عدلك, رب]",عدلك رب,"[عدلك, رب]"
2,1,0,0,إلا قرقعة ودوشة قارفينا ببيتي قال فن قال,24/01/2020,20:02,"[قرقعة, ودوشة, قارفينا, ببيتي, قال, فن, قال]",قرقعه ودوشه قارفينا بيتي قال فن قال,"[قرقعه, ودوشه, قارفينا, بيتي, قال, فن, قال]"
3,1,0,0,٦ شهور محبورة اعيش ببيتي والبوليفارد يردح على ...,21/01/2020,21:03,"[٦, شهور, محبورة, اعيش, ببيتي, والبوليفارد, ير...",٦ شهور محبوره اعيش بيتي والبوليفارد يردح راسي...,"[٦, شهور, محبوره, اعيش, بيتي, والبوليفارد, يرد..."
4,0,0,0,في شويه معرصين حطين الصورة دي انا مش مطالبه ا...,19/01/2020,21:59,"[شويه, معرصين, حطين, الصورة, دي, انا, مش, مطال...",شويه معرصين حطين الصوره دي انا مش مطالبه اقعد...,"[شويه, معرصين, حطين, الصوره, دي, انا, مش, مطال..."
...,...,...,...,...,...,...,...,...,...
876,1,0,0,أتمنى منك عرض معاناة منطقة جازان في توسعة مل...,12/05/2012,19:35,"[أتمنى, منك, عرض, معاناة, منطقة, جازان, توسعة,...",اتمني منك عرض معاناه منطقه جازان توسعه ملعب م...,"[اتمني, منك, عرض, معاناه, منطقه, جازان, توسعه,..."
877,0,0,0,صدقني أحس إن حطين بالدرجة الثانية ماله حس و ...,12/01/2012,20:46,"[صدقني, أحس, حطين, بالدرجة, الثانية, ماله, حس,...",صدقني احس حطين بالدرجه الثانيه ماله حس و ماكا...,"[صدقني, احس, حطين, بالدرجه, الثانيه, ماله, حس,..."
878,0,0,0,مع ان جيرانا من حطين ولوبية وصفورية والمرين ...,18/09/2012,00:53,"[ان, جيرانا, حطين, ولوبية, وصفورية, والمرين, ف...",ان جيرانا حطين ولوبيه وصفوريه والمرين فقط علي...,"[ان, جيرانا, حطين, ولوبيه, وصفوريه, والمرين, ف..."
879,0,0,0,اليوم الطائي يحل على حطين ضيف ولسان حال العشاق...,09/12/2012,13:10,"[اليوم, الطائي, يحل, حطين, ضيف, ولسان, حال, ال...",اليوم الطائي يحل حطين ضيف ولسان حال العشاق لر...,"[اليوم, الطائي, يحل, حطين, ضيف, ولسان, حال, ال..."


In [232]:
#Delete Empty Rows or Rows that contain only one word or a letter 
df.drop(df[df['normalized'].apply(len) < 5].index, inplace=True)

C:\Users\jwoha\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [233]:
#Delete Retweeted Tweets 
df.drop(df[df['normalized'].str.contains('Retweeted')].index, inplace=True)

In [234]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 808 entries, 0 to 880
Data columns (total 9 columns):
replies       808 non-null int64
retweets      808 non-null int64
favorites     808 non-null int64
text          808 non-null object
day           808 non-null object
time          808 non-null object
tokens        808 non-null object
normalized    808 non-null object
tokenized     808 non-null object
dtypes: int64(3), object(6)
memory usage: 44.2+ KB


In [235]:
export_csv = df.to_csv (r'moreMoreCleanedVersion.csv', index = None, header=True, encoding='utf-8-sig')

In [236]:
import mazajak_api as mz


In [253]:
tweets_list = df['normalized'].tolist()

In [254]:
print(tweets_list[0])

 وماذا ازعاج يزيد سته اشهر لاحياء سكنيه والاف السكان ومئات الاسر وعشرات المساجد جاورها معاناه وماساه وظلم يزال التفات رفع استجابه


In [255]:
predictionsList= mz.predict_list(tweets_list);
i=0
for predict in predictionsList:
    print("Tweet: "+ tweets_list[i]);
    print("Prediction: "+ predict);
    i = i + 1;

Tweet:  وماذا ازعاج يزيد سته اشهر لاحياء سكنيه والاف السكان ومئات الاسر وعشرات المساجد جاورها معاناه وماساه وظلم يزال التفات رفع استجابه
Prediction: negative
Tweet:  عدلك رب
Prediction: positive
Tweet:  قرقعه ودوشه قارفينا بيتي قال فن قال
Prediction: negative
Tweet:  ٦ شهور محبوره اعيش بيتي والبوليفارد يردح راسي مشاركنا بيتنا اش هالتجني وعدم احترام حرمات الناس اله يخوي روسهم مثل اخوا روسنا بسماعاتهم طوال اليوم
Prediction: negative
Tweet:  شويه معرصين حطين الصوره دي انا مش مطالبه اقعد اشرح معاناه الي تعبنا واحنا بنشرحها وانتوا خنزير ناطقه مبتفهمش اي حد حاط الصوره دي احب اقوله اسكت ياعر ص ء
Prediction: negative
Tweet:  حسبي اله ونعم الوكيل عدلك رب العالمين
Prediction: positive
Tweet:  حلول المرور مشاكل يقفلون اليوتيرن الي اشاره تركي الاول يقال بيخفون الزحمه والحاصل زحمه متده حطين ودي اوقف السياره وارجع البيت مشي
Prediction: negative
Tweet:  عدلك رب المسؤول الازعاج و السماعات المسعوره اله يجعل حوبه ازعاج الحي منازلهم تعاداه صحته و رزقه حسبي اله ونعم الوكيل
Prediction: negative
Tweet:  اتم

In [256]:
predictionsList

['negative',
 'positive',
 'negative',
 'negative',
 'negative',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'neutral',
 'negative',
 'negative',
 'neutral',
 'neutral',
 'positive',
 'negative',
 'neutral',
 'neutral',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'positive',
 'negative',
 'positive',
 'positive',
 'negative',
 'negative',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'neutral',
 'negative',
 'negative',
 'negative',
 'positive',
 'negative',
 'negative',
 'neutral',
 'positive',
 'negative',
 'negative',
 'negative',
 'neutral',
 'negative',
 'negative',
 'neutral',
 'negative',
 'negative',
 'negative',
 'neutral',
 'negative',
 'positive',
 'positive',
 'negative',
 'neutral',
 'negative',
 'negative',
 'negative',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'neutral',
 'neutral',
 'negative',
 'negative',
 'negative',
 'neutral',


In [257]:
print(len(predictionsList))

808


In [258]:
df['predictions'] = predictionsList

In [259]:
df

,replies,retweets,favorites,text,day,time,tokens,normalized,tokenized,predictions
0,0,0,0,وماذا عن إزعاج ما يزيد على ستة أشهر لأحياء سكن...,02/08/2020,09:16,"[وماذا, إزعاج, يزيد, ستة, أشهر, لأحياء, سكنية,...",وماذا ازعاج يزيد سته اشهر لاحياء سكنيه والاف ...,"[وماذا, ازعاج, يزيد, سته, اشهر, لاحياء, سكنيه,...",negative
1,0,0,0,عدلك يا رب,02/04/2020,22:03,"[عدلك, رب]",عدلك رب,"[عدلك, رب]",positive
2,1,0,0,إلا قرقعة ودوشة قارفينا ببيتي قال فن قال,24/01/2020,20:02,"[قرقعة, ودوشة, قارفينا, ببيتي, قال, فن, قال]",قرقعه ودوشه قارفينا بيتي قال فن قال,"[قرقعه, ودوشه, قارفينا, بيتي, قال, فن, قال]",negative
3,1,0,0,٦ شهور محبورة اعيش ببيتي والبوليفارد يردح على ...,21/01/2020,21:03,"[٦, شهور, محبورة, اعيش, ببيتي, والبوليفارد, ير...",٦ شهور محبوره اعيش بيتي والبوليفارد يردح راسي...,"[٦, شهور, محبوره, اعيش, بيتي, والبوليفارد, يرد...",negative
4,0,0,0,في شويه معرصين حطين الصورة دي انا مش مطالبه ا...,19/01/2020,21:59,"[شويه, معرصين, حطين, الصورة, دي, انا, مش, مطال...",شويه معرصين حطين الصوره دي انا مش مطالبه اقعد...,"[شويه, معرصين, حطين, الصوره, دي, انا, مش, مطال...",negative
...,...,...,...,...,...,...,...,...,...,...
876,1,0,0,أتمنى منك عرض معاناة منطقة جازان في توسعة مل...,12/05/2012,19:35,"[أتمنى, منك, عرض, معاناة, منطقة, جازان, توسعة,...",اتمني منك عرض معاناه منطقه جازان توسعه ملعب م...,"[اتمني, منك, عرض, معاناه, منطقه, جازان, توسعه,...",neutral
877,0,0,0,صدقني أحس إن حطين بالدرجة الثانية ماله حس و ...,12/01/2012,20:46,"[صدقني, أحس, حطين, بالدرجة, الثانية, ماله, حس,...",صدقني احس حطين بالدرجه الثانيه ماله حس و ماكا...,"[صدقني, احس, حطين, بالدرجه, الثانيه, ماله, حس,...",negative
878,0,0,0,مع ان جيرانا من حطين ولوبية وصفورية والمرين ...,18/09/2012,00:53,"[ان, جيرانا, حطين, ولوبية, وصفورية, والمرين, ف...",ان جيرانا حطين ولوبيه وصفوريه والمرين فقط علي...,"[ان, جيرانا, حطين, ولوبيه, وصفوريه, والمرين, ف...",negative
879,0,0,0,اليوم الطائي يحل على حطين ضيف ولسان حال العشاق...,09/12/2012,13:10,"[اليوم, الطائي, يحل, حطين, ضيف, ولسان, حال, ال...",اليوم الطائي يحل حطين ضيف ولسان حال العشاق لر...,"[اليوم, الطائي, يحل, حطين, ضيف, ولسان, حال, ال...",positive


In [260]:
export_csv = df.to_csv (r'HittenTweetsPhase3WithPredictions.csv', index = None, header=True, encoding='utf-8-sig')

In [261]:
df =df.drop(columns="predictions")

In [262]:
df

,replies,retweets,favorites,text,day,time,tokens,normalized,tokenized
0,0,0,0,وماذا عن إزعاج ما يزيد على ستة أشهر لأحياء سكن...,02/08/2020,09:16,"[وماذا, إزعاج, يزيد, ستة, أشهر, لأحياء, سكنية,...",وماذا ازعاج يزيد سته اشهر لاحياء سكنيه والاف ...,"[وماذا, ازعاج, يزيد, سته, اشهر, لاحياء, سكنيه,..."
1,0,0,0,عدلك يا رب,02/04/2020,22:03,"[عدلك, رب]",عدلك رب,"[عدلك, رب]"
2,1,0,0,إلا قرقعة ودوشة قارفينا ببيتي قال فن قال,24/01/2020,20:02,"[قرقعة, ودوشة, قارفينا, ببيتي, قال, فن, قال]",قرقعه ودوشه قارفينا بيتي قال فن قال,"[قرقعه, ودوشه, قارفينا, بيتي, قال, فن, قال]"
3,1,0,0,٦ شهور محبورة اعيش ببيتي والبوليفارد يردح على ...,21/01/2020,21:03,"[٦, شهور, محبورة, اعيش, ببيتي, والبوليفارد, ير...",٦ شهور محبوره اعيش بيتي والبوليفارد يردح راسي...,"[٦, شهور, محبوره, اعيش, بيتي, والبوليفارد, يرد..."
4,0,0,0,في شويه معرصين حطين الصورة دي انا مش مطالبه ا...,19/01/2020,21:59,"[شويه, معرصين, حطين, الصورة, دي, انا, مش, مطال...",شويه معرصين حطين الصوره دي انا مش مطالبه اقعد...,"[شويه, معرصين, حطين, الصوره, دي, انا, مش, مطال..."
...,...,...,...,...,...,...,...,...,...
876,1,0,0,أتمنى منك عرض معاناة منطقة جازان في توسعة مل...,12/05/2012,19:35,"[أتمنى, منك, عرض, معاناة, منطقة, جازان, توسعة,...",اتمني منك عرض معاناه منطقه جازان توسعه ملعب م...,"[اتمني, منك, عرض, معاناه, منطقه, جازان, توسعه,..."
877,0,0,0,صدقني أحس إن حطين بالدرجة الثانية ماله حس و ...,12/01/2012,20:46,"[صدقني, أحس, حطين, بالدرجة, الثانية, ماله, حس,...",صدقني احس حطين بالدرجه الثانيه ماله حس و ماكا...,"[صدقني, احس, حطين, بالدرجه, الثانيه, ماله, حس,..."
878,0,0,0,مع ان جيرانا من حطين ولوبية وصفورية والمرين ...,18/09/2012,00:53,"[ان, جيرانا, حطين, ولوبية, وصفورية, والمرين, ف...",ان جيرانا حطين ولوبيه وصفوريه والمرين فقط علي...,"[ان, جيرانا, حطين, ولوبيه, وصفوريه, والمرين, ف..."
879,0,0,0,اليوم الطائي يحل على حطين ضيف ولسان حال العشاق...,09/12/2012,13:10,"[اليوم, الطائي, يحل, حطين, ضيف, ولسان, حال, ال...",اليوم الطائي يحل حطين ضيف ولسان حال العشاق لر...,"[اليوم, الطائي, يحل, حطين, ضيف, ولسان, حال, ال..."


In [267]:
export_csv = df.to_csv (r'HittenTweetsPhase3WithoutPredictions.csv', index = None, header=True, encoding='utf-8-sig')